In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
from matplotlib import pyplot as plt
import mpl_axes_aligner

import seaborn as sns
import zipfile
from cycler import cycler

pd.options.display.float_format = '{:.4f}'.format


# global plotting settings
plt.style.use(['ggplot'])

default_cycler = (plt.rcParams['axes.prop_cycle'][:4]  +
                  cycler(linestyle=['-', '--', ':', '-.']))

plt.rcParams.update({'axes.grid' : True, 
                     'axes.facecolor' : 'white', 
                     'axes.edgecolor' : '.15',
                     'axes.labelcolor' : 'black',
                     'yaxis.labellocation' : 'center',
                     'lines.marker' : '.',
                     'lines.linestyle' : 'None',
                     'grid.color' : '.8',
                     'axes.prop_cycle' : default_cycler,
                     'figure.figsize' : (10*1.8, 5.65*1.8),
                     'mathtext.default' : 'regular'
                    })

# plt.rc('axes', prop_cycle=default_cycler)

data_dir = f'/home/otto/data/atmos-flux-data/methane-gapfill/data_dir/'

def read_l3(filepath, datecol='DATE'):
    df = (pd.read_csv(filepath, header=0, low_memory=False)
      .replace(-9999, np.nan).replace(99999, np.nan)
      .assign(date = lambda x: pd.to_datetime(x[datecol]))
      .set_index('date', drop=False)
     )
    df.index = df.index.set_names(None)
    return df

filepath = '/home/otto/data/atmos-flux-data/processed/ATMOS_L3_2023-04-25.csv'
l3 = read_l3(filepath)

predictors = 'baseflux'
filepath = f'/home/otto/data/atmos-flux-data/methane-gapfill/data_dir/ATMOS_20230425_{predictors}/gapfilled/rf_predictors_laplace.csv'
rf = read_l3(filepath).add_suffix('_RF')

filepath = f'/home/otto/data/atmos-flux-data/methane-gapfill/data_dir/ATMOS_20230425_{predictors}/gapfilled/ann_predictors_laplace.csv'
ann = read_l3(filepath).add_suffix('_ANN')

l3 = pd.concat([l3, rf.loc[:,'FCH4_F_RF':], ann.loc[:,'FCH4_F_ANN':]], axis=1)
# l3.to_csv('./output/csv/ATMOS_L3_{datetime.now().strftime("%Y%m%d")}_ch4gapfilled.csv', index = False)

platt_constant = rf['FCH4_F_UNCERTAINTY_RF'] / rf.loc[:, [f'FCH4_F{i}_RF' for i in range(1,11)]].std(axis=1)
# (platt_constant)

In [3]:
# output L3 fluxes with ch4 gap filling

filepath = '/home/otto/data/atmos-flux-data/processed/ATMOS_L3_2023-04-25_fluxgapfill.csv'
l3 = read_l3(filepath)

predictors = 'baseflux'
filepath = f'/home/otto/data/atmos-flux-data/methane-gapfill/data_dir/ATMOS_20230425_{predictors}/gapfilled/rf_predictors_laplace.csv'
rf = read_l3(filepath).add_suffix('_RF')

filepath = f'/home/otto/data/atmos-flux-data/methane-gapfill/data_dir/ATMOS_20230425_{predictors}/gapfilled/ann_predictors_laplace.csv'
ann = read_l3(filepath).add_suffix('_ANN')

l3 = pd.concat([l3, rf.loc[:,'FCH4_F_RF':], ann.loc[:,'FCH4_F_ANN':]], axis=1)
l3.to_csv(f'./output/csv/ATMOS_L3_{datetime.now().strftime("%Y-%m-%d")}_ch4gapfilled.csv', index = False)

In [167]:
# format to FLUXNET
def save_fluxnet(filepath, datecol='DATE'):
    df = (pd.read_csv(filepath, header=0, low_memory=False)
      .replace(np.nan, -9999).replace(99999, -9999)
      .rename(columns = {'L': 'MO_LENGTH'
                         }, errors = 'raise')
      .assign(date = lambda x: pd.to_datetime(x[datecol]))
      .set_index('date', drop=False)
     )
    df.index = df.index.set_names(None)
    df.columns = df.columns.str.replace('qc_LE', 'LE_SSITC_TEST')
    df.columns = df.columns.str.replace('un_LE', 'LE_UNCORR')
    df.columns = df.columns.str.replace('qc_H', 'H_SSITC_TEST')
    df.columns = df.columns.str.replace('un_H', 'H_UNCORR')
    df.columns = df.columns.str.replace('qc_co2_flux', 'FC_SSITC_TEST')
    df.columns = df.columns.str.replace('un_co2_flux', 'FC_UNCORR')
    df.columns = df.columns.str.replace('co2_flux', 'FC')
    df.columns = df.columns.str.replace('qc_ch4_flux', 'FCH4_SSITC_TEST')
    df.columns = df.columns.str.replace('un_ch4_flux', 'FCH4_UNCORR')
    df.columns = df.columns.str.replace('ch4_flux', 'FCH4')
    df.columns = df.columns.str.replace('qc_Tau', 'TAU_SSITC_TEST')
    df.columns = df.columns.str.replace('un_Tau', 'TAU_UNCORR')
    df.columns = df.columns.str.replace('Tau', 'TAU')
    df.columns = df.columns.str.replace('Reco', 'RECO')
    df.columns = df.columns.str.replace('Reco_', 'RECO_')
    df.columns = df.columns.str.replace('RN', 'NETRAD')
    df.columns = df.columns.str.replace('Precip', 'P')
    df.columns = df.columns.str.replace('sonic_temperature', 'T_SONIC')
    df.columns = df.columns.str.replace('max_wind_speed', 'WS_MAX')
    df.columns = df.columns.str.replace('wind_speed', 'WS')
    df.columns = df.columns.str.replace('Tsoil', 'TS')
    df.to_csv(f'./output/csv/ATMOS_L3_{datetime.now().strftime("%Y-%m-%d")}_fluxnet.csv', index = False)
    return df

filepath = f'./output/csv/ATMOS_L3_{datetime.now().strftime("%Y-%m-%d")}_ch4gapfilled.csv'
fluxnet = save_fluxnet(filepath)
# df = pd.read_csv('./output/csv/ATMOS_L3_{datetime.now().strftime("%Y%m%d")}_fluxnet.csv')

In [144]:
fluxnet['WS']

2022-05-24 10:30:00       2.4297
2022-05-24 11:00:00       2.4546
2022-05-24 11:30:00   -9999.0000
2022-05-24 12:00:00   -9999.0000
2022-05-24 12:30:00       3.2639
                         ...    
2023-04-06 09:30:00       3.3781
2023-04-06 10:00:00       3.8018
2023-04-06 10:30:00       4.0137
2023-04-06 11:00:00       3.8492
2023-04-06 11:30:00       3.7664
Name: WS, Length: 15219, dtype: float64

In [166]:
fluxnet.loc[:, fluxnet.columns.str.contains('co2_')]

,co2_strg,co2_v.adv,co2_molar_density,co2_mole_fraction,co2_mixing_ratio,co2_time_lag,co2_def_timelag,co2_scf,co2_spikes,co2_var,w.co2_cov,co2_mean,co2_signal_strength_7500_mean
2022-05-24 10:30:00,-9999.0000,-0.0000,17.0894,413.6080,417.4470,0.0000,0.0000,1.1139,8.0000,0.0110,-0.0179,412.2520,103.5530
2022-05-24 11:00:00,-0.0951,-0.0000,17.0439,412.5710,416.3670,0.0000,0.0000,1.1141,6.0000,0.0087,-0.0143,410.8380,103.5560
2022-05-24 11:30:00,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000
2022-05-24 12:00:00,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000
2022-05-24 12:30:00,-9999.0000,-0.0000,17.0624,416.6810,420.5070,0.0000,0.0000,1.1227,18.0000,0.0162,-0.0268,415.2400,103.5120
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-06 09:30:00,-0.0410,0.0000,18.3725,425.2970,426.7400,0.0000,0.0000,1.1241,8.0000,0.0028,-0.0106,425.7380,102.9490
2023-04-06 10:00:00,-0.0890,-0.0000,18.2943,424.3680,425.8030,0.1000,0.0000,1.1291,19.0000,0.0034,-0.0113,423.8570,102.9550
2023-04-06 10:30:00,-0.0785,0.0000,18.2421,423.5470,424.9590,0.0000,0.0000,1.1316,14.0000,0.0031,-0.0140,423.2460,102.9560
2023-04-06 11:00:00,-0.0345,0.0000,18.1738,423.1860,424.6020,0.0000,0.0000,1.1296,20.0000,0.0038,-0.0146,421.5470,102.9620


In [56]:
fluxnet.loc[:, fluxnet.columns.str.contains('co2_flux')].iloc[np.where(fluxnet.qc_co2_flux == 2)[0]]

,co2_flux,qc_co2_flux,un_co2_flux
2022-05-25 09:30:00,-9999.0000,2.0000,-0.1852
2022-05-25 12:00:00,-9999.0000,2.0000,-1.0082
2022-05-25 17:00:00,-9999.0000,2.0000,-6.6699
2022-05-25 18:30:00,-9999.0000,2.0000,0.7907
2022-05-26 00:30:00,-9999.0000,2.0000,3.6211
...,...,...,...
2023-04-04 18:30:00,-9999.0000,2.0000,-1.3374
2023-04-04 20:30:00,-9999.0000,2.0000,-0.9477
2023-04-05 06:30:00,-9999.0000,2.0000,-0.5323
2023-04-05 11:00:00,-9999.0000,2.0000,0.4704


In [162]:
fluxnet.loc[:, fluxnet.columns.str.contains('un_')]

,un_TAU,un_H,un_LE,un_FC,un_h2o_flux,un_FCH4
2022-05-24 10:30:00,-0.1205,80.3292,116.3100,-17.9290,2.6226,-0.0362
2022-05-24 11:00:00,-0.1226,47.4885,114.9180,-14.3493,2.5913,-0.0492
2022-05-24 11:30:00,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000
2022-05-24 12:00:00,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000
2022-05-24 12:30:00,-0.1891,164.1820,205.2780,-26.7710,4.6392,-0.1480
...,...,...,...,...,...,...
2023-04-06 09:30:00,-0.1597,171.6720,40.8406,-10.6312,0.9106,-0.0595
2023-04-06 10:00:00,-0.1594,193.4960,44.0458,-11.3171,0.9827,-0.0558
2023-04-06 10:30:00,-0.2652,225.3050,57.1297,-13.9822,1.2751,-0.0744
2023-04-06 11:00:00,-0.1906,251.7150,60.7415,-14.5996,1.3568,-0.0744


In [161]:
filepath = '/home/otto/data/atmos-flux-data/processedEP/20230305-rotate-all/eddypro_atmos_fluxnet_2023-03-05T224639_exp.csv'
df = pd.read_csv(filepath, header = 0, low_memory=False)
# df.loc[:, df.columns.str.contains('FC')]
df.loc[:, df.columns.str.contains('FC_')].iloc[np.where(df['FC_SSITC_TEST'] == 1)[0][0:20]]

,FC_NR,FC_RANDUNC_HF,FC_VADV,FC_UNCORR,FC_STAGE1,FC_STAGE2,FC_SCF,FC_CORRDIFF,FC_NSR,FC_SS,FC_SS_TEST,FC_SSITC_TEST
8,18000,-9999,-0.0000,-30.1633,-33.7434,-21.4282,1.1187,0.0194,1.0840,1,1,1
9,18000,-9999,0.0000,-25.1759,-28.1921,-16.7639,1.1198,0.0229,1.1794,1,1,1
10,18000,-9999,-0.0000,-20.5638,-23.0141,-14.3449,1.1192,0.0167,2.0466,1,1,1
12,18000,-9999,0.0000,-18.6659,-20.9046,-13.5366,1.1199,0.0045,0.9604,3,1,1
16,18000,-9999,0.0000,-4.5691,-5.0714,-3.2140,1.1099,0.0220,2.2384,21,2,1
17,18000,-9999,-0.0000,2.8975,3.2680,2.7338,1.1278,0.0119,1.5617,22,2,1
18,18000,-9999,-0.0000,8.5786,9.6320,8.4970,1.1228,0.0166,1.1531,35,3,1
19,17999,-9999,0.0000,5.9392,6.6784,6.4763,1.1245,0.0015,0.8661,8,1,1
20,18000,-9999,0.0000,6.9925,7.8646,7.4889,1.1247,0.0069,0.6011,13,1,1
21,17999,-9999,-0.0000,7.5092,8.4402,7.6900,1.1240,0.0064,1.0218,7,1,1


In [ ]:
filepath = '/home/otto/data/atmos-flux-data/processed/ATMOS_L3_2023-04-25_fluxgapfill.csv'
df = (pd.read_csv(filepath, header = 0, low_memory=False)
            .assign(date = lambda x: pd.to_datetime(x['DATE']))
      .set_index('date', drop=False)
     )
df.index = df.index.set_names(None)

df.loc[df.values == 99999, df.loc[:, (df == 99999).any()].columns]
# df.loc['2023-03-31 23:00:00':'2023-04-02']
# df.loc[:, 'T10'].loc[df.values == 99999]

## Annual metrics

In [2]:
cols = ['TA_EP','T60','Tsoil_mean','SWC_mean','Precip']

annualmet = l3[cols].agg(['mean', 'min', 'max'])
annualmet.loc['sum', 'Precip'] = l3.loc[:, 'Precip'].sum()
annualmet.loc['std', ['TA_EP', 'T60', 'Tsoil_mean', 'SWC_mean']] = l3.loc[:, ['TA_EP', 'T60', 'Tsoil_mean', 'SWC_mean']].std()
# mean and standard deviation of TA_EP, mean and ReddyProc unc of co2_flux, mean and gap-fill unc of FCH4

# annualmet.loc['uncertainty', :] = annualmet.std()
# annualmet.loc['uncertainty', ['NEE_f','co2_flux','FCH4_f','FCH4_F_RF']] = np.NaN

annualmet

,TA_EP,T60,Tsoil_mean,SWC_mean,Precip
mean,12.5219,12.0268,11.5006,0.3027,0.1156
min,-23.1185,-18.4000,1.6865,0.1573,0.0000
max,36.6963,34.8000,22.7450,0.4286,67.3000
sum,NaN,NaN,NaN,NaN,810.8000
std,11.7231,10.8633,7.2354,0.0819,NaN


In [3]:
cols = [f'FCH4_F{i}_RF' for i in range(1,11)]
model_sum = l3.loc[:, cols].sum(axis=0) * 1800 / 1000 / 1000 * 12.01
ch4_budget = [model_sum.mean(), model_sum.std() * 1.4210]
ch4_budget
# model_sum

[-0.3816044614926144, 3.0614157890823375]

In [4]:
ustars = l3.loc[:, 'ustar_thr'].unique()
ustars

array([0.14981061, 0.194696  ])

In [5]:
cols = ['ch4_flux', 'co2_flux', 'h2o_flux', 'NEE_f', 'FCH4_F_RF']
data_coverage = l3[cols].count()/l3.shape[0]
data_coverage

ch4_flux    0.3674
co2_flux    0.5446
h2o_flux    0.5681
NEE_f       1.0000
FCH4_F_RF   1.0000
dtype: float64

In [6]:
umoltogC = 1800 / 1000 / 1000 * 12.01 # [umol m-2 s-1] * (s 30min-1) / (umol mmol-1) / (mmol mol-1) * (gC molC-1)
umoltomgC = 1800 / 1000 * 12.01 # [umol m-2 s-1] * (s 30min-1) / (umol mmol-1) * (mgC mmolC-1)
cols = ['ch4_flux','FCH4_f', 'FCH4_F_ANN', 'FCH4_F_RF', 'co2_flux', 'NEE_f', 'GPP_f', 'Reco']
budget = l3[cols] * umoltogC
budget = budget.sum(axis=0)
budget['H_f'] = l3['H_f'].sum() / 1000
budget['LE_f'] = l3['LE_f'].sum() / 1000
# budget['NEE_%grow'] = l3.loc['2022-06':'2022-09-01', 'NEE_f'].sum()
budget['FCH4_uptake'] = l3.loc[l3['FCH4_F_RF']<0, 'FCH4_F_RF'].sum() * umoltogC
budget['FCH4_emission'] = l3.loc[l3['FCH4_F_RF']>0, 'FCH4_F_RF'].sum() * umoltogC
budget['FCH4_uptake%'] = budget['FCH4_uptake'] / (abs(budget['FCH4_uptake']) + abs(budget['FCH4_emission']))
budget['FCH4_emission%'] = budget['FCH4_emission'] / (abs(budget['FCH4_uptake']) + abs(budget['FCH4_emission']))
budget['FCH4_co2eq'] = budget['FCH4_F_RF'] / 12.01 * 16.04 * 45
budget['FCH4_co2eq_up'] = budget['FCH4_uptake'] / 12.01 * 16.04 * 45
budget['FCH4_co2eq_emit'] = budget['FCH4_emission'] / 12.01 * 16.04 * 45
budget['NEE_co2eq'] = budget['NEE_f'] / 12.01 * 44.01 # gC to molC, molC to gCO2
budget['rad_balance'] = (budget['FCH4_co2eq_up']) + (budget['FCH4_co2eq_emit']) + budget['NEE_co2eq']
# pd.concat([budget, pd.DataFrame({'rad_balance': budget['FCH4_F_RF'] * 45 + budget['NEE_f'], 
#                                  'ch4_to_0rad': budget['NEE_f'] / 45})])
budget['ch4_to_0rad'] = -(budget['NEE_co2eq'] / 45 / 16.04 * 12.01) # gCO2eq to gCH4, gCH4 to molC, molC to gC  
budget

ch4_flux            -0.3512
FCH4_f              -0.5024
FCH4_F_ANN        -550.6893
FCH4_F_RF           -0.3816
co2_flux          -356.7822
NEE_f              -18.5232
GPP_f             1804.8325
Reco              1786.3092
H_f                199.8094
LE_f               859.9817
FCH4_uptake         -1.8768
FCH4_emission        1.4952
FCH4_uptake%        -0.5566
FCH4_emission%       0.4434
FCH4_co2eq         -22.9344
FCH4_co2eq_up     -112.7970
FCH4_co2eq_emit     89.8626
NEE_co2eq          -67.8774
rad_balance        -90.8118
ch4_to_0rad          1.1294
dtype: float64

In [8]:
weekly = l3.groupby(l3.date.dt.to_period('W')).agg(TA_EP_mean = ('TA_EP', np.mean),
                                                  Tsoil_mean=('Tsoil_mean', np.mean), 
                                                  SWC_mean=('SWC_mean', np.mean), 
                                                  FCH4_F_RF_mean=('FCH4_F_RF', lambda x: np.mean(x * 1000)), 
                                                   NEE_f_mean=('NEE_f', np.mean), 
                                                   GPP_f_mean=('GPP_f', np.mean), 
                                                   Reco_mean=('GPP_f', np.mean),
                                                  FCH4_F_RF_sum=('FCH4_F_RF', lambda x: np.sum(x * 1800 / 1000 * 12.01)), # mgC m-2
                                                   NEE_f_sum=('NEE_f', lambda x: np.sum(x * 1800 / 1000 / 1000 * 12.01)), # gC m-2
                                                   GPP_f_sum=('GPP_f', lambda x: np.sum(x * 1800 / 1000 / 1000 * 12.01)), # gC m-2 
                                                   Reco_sum=('Reco', lambda x: np.sum(x * 1800 / 1000 / 1000 * 12.01)), # gC m-2
                                                  H_f_sum=('H_f', lambda x: np.sum(x / 1000)), # W m-2 to kW m-2
                                                  LE_f_sum=('LE_f', lambda x: np.sum(x / 1000)), # W m-2 to kW m-2
                                                   Precip_sum=('Precip', np.sum)
                                                   ).reset_index().set_index('date')
# weekly.index = weekly.index.set_names(None)

monthly = l3.groupby(l3.date.dt.to_period('M')).agg(TA_EP_mean = ('TA_EP', np.mean),
                                                  Tsoil_mean=('Tsoil_mean', np.mean), 
                                                  SWC_mean=('SWC_mean', np.mean), 
                                                  FCH4_F_RF_mean=('FCH4_F_RF', lambda x: np.mean(x * 1000)), # nmol m-2 
                                                   NEE_f_mean=('NEE_f', np.mean), 
                                                   GPP_f_mean=('GPP_f', np.mean), 
                                                   Reco_mean=('GPP_f', np.mean),
                                                  FCH4_F_RF_sum=('FCH4_F_RF', lambda x: np.sum(x * 1800 / 1000 * 12.01)), # mgC m-2
                                                   NEE_f_sum=('NEE_f', lambda x: np.sum(x * 1800 / 1000 / 1000 * 12.01)), # gC m-2
                                                   GPP_f_sum=('GPP_f', lambda x: np.sum(x * 1800 / 1000 / 1000 * 12.01)), # gC m-2 
                                                   Reco_sum=('Reco', lambda x: np.sum(x * 1800 / 1000 / 1000 * 12.01)), # gC m-2
                                                  H_f_sum=('H_f', lambda x: np.sum(x / 1000)), # W m-2 to kW m-2
                                                  LE_f_sum=('LE_f', lambda x: np.sum(x / 1000)), # W m-2 to kW m-2
                                                   Precip_sum=('Precip', np.sum)
                                                   ).reset_index().set_index('date')
# monthly.index = monthly.index.set_names(None)

# # cols = ['FC', 'NEE_f', 'Reco', 'GPP_f', 'FCH4', 'FCH4_f', 'H_f'] # Fluxnet variables
# cols = ['co2_flux', 'NEE_f', 'Reco', 'GPP_f', 'ch4_flux', 'FCH4_f', 'FCH4_F_RF', 'FCH4_F_ANN', 'LE', 'LE_f','H_f'] # Full output variables

# dailysum = l3.loc[:, cols].groupby(pd.Grouper(freq='1D')).sum() * 1800 / 1000
# dailysum.loc[:, 'LE_f'] = l3.loc[:, ['LE_f']].groupby(pd.Grouper(freq='1D')).sum()

# daily = l3.loc[:, cols].groupby(l3.index.floor('D')).mean()
# # daily[['FCH4', 'FCH4_f', 'FCH4_F_ANN']]
# # daily.loc[:,'FCH4']
# daily

daily = l3.groupby(l3.date.dt.to_period('D')).agg(TA_EP_mean = ('TA_EP', np.mean),
                                                  Tsoil_mean=('Tsoil_mean', np.mean), 
                                                  SWC_mean=('SWC_mean', np.mean), 
                                                  FCH4_F_RF_mean=('FCH4_F_RF', lambda x: np.mean(x * 1000)), # nmol m-2 
                                                   NEE_f_mean=('NEE_f', np.mean), 
                                                   GPP_f_mean=('GPP_f', np.mean), 
                                                   Reco_mean=('GPP_f', np.mean),
                                                  FCH4_F_RF_sum=('FCH4_F_RF', lambda x: np.sum(x * 1800 / 1000 * 12.01)), # mgC m-2
                                                   NEE_f_sum=('NEE_f', lambda x: np.sum(x * 1800 / 1000 / 1000 * 12.01)), # gC m-2
                                                   GPP_f_sum=('GPP_f', lambda x: np.sum(x * 1800 / 1000 / 1000 * 12.01)), # gC m-2 
                                                   Reco_sum=('Reco', lambda x: np.sum(x * 1800 / 1000 / 1000 * 12.01)), # gC m-2
                                                  H_f_sum=('H_f', lambda x: np.sum(x / 1000)), # W m-2 to kW m-2
                                                  LE_f_sum=('LE_f', lambda x: np.sum(x / 1000)), # W m-2 to kW m-2
                                                   Precip_sum=('Precip', np.sum)
                                                 ).reset_index().set_index('date')

hourly = l3.groupby(l3.date.dt.to_period('H')).agg(TA_EP_mean = ('TA_EP', np.mean),
                                                  Tsoil_mean=('Tsoil_mean', np.mean), 
                                                  SWC_mean=('SWC_mean', np.mean), 
                                                  FCH4_F_RF_mean=('FCH4_F_RF', lambda x: np.mean(x * 1000)), # nmol m-2 
                                                   NEE_f_mean=('NEE_f', np.mean), 
                                                   GPP_f_mean=('GPP_f', np.mean), 
                                                   Reco_mean=('GPP_f', np.mean),
                                                  FCH4_F_RF_sum=('FCH4_F_RF', lambda x: np.sum(x * 1800 / 1000 * 12.01)), # mgC m-2
                                                   NEE_f_sum=('NEE_f', lambda x: np.sum(x * 1800 / 1000 / 1000 * 12.01)), # gC m-2
                                                   GPP_f_sum=('GPP_f', lambda x: np.sum(x * 1800 / 1000 / 1000 * 12.01)), # gC m-2 
                                                   Reco_sum=('Reco', lambda x: np.sum(x * 1800 / 1000 / 1000 * 12.01)), # gC m-2
                                                  H_f_sum=('H_f', lambda x: np.sum(x / 1000)), # W m-2 to kW m-2
                                                  LE_f_sum=('LE_f', lambda x: np.sum(x / 1000)), # W m-2 to kW m-2
                                                   Precip_sum=('Precip', np.sum)
                                                 ).reset_index().set_index('date')

In [9]:
daily.loc['2022-11':'2023-02','LE_f_sum'].max()

2.1358680826686793

In [10]:
output = monthly.drop(columns=['GPP_f_mean', 'Reco_mean'])
# output.to_csv('./output/thesis/monthly_metrics.csv', index = True)